In [13]:
import pandas as pd
import numpy as np
# import os
import jdc

In [20]:
class Mech():
    def __init__(self, chromosomes, sample_id):
        usecols = ["Chr", 
                   "Start", "End", "Ref", "Alt", "Func.refGene", "Otherinfo10", "Otherinfo11"]
        dtype = {"Chr":"category",
          "Start":np.int32,
          "End":np.int32,
          "Ref":"category",
          "Alt":"category",
          "Func.refGene":"category",
          "Otherinfo10":"category",
          "Otherinfo11":"string"}
        self.chromosomes = chromosomes
        self.sample_id = sample_id
        self.df = pd.concat(map(pd.read_csv, 
                    ["chr{}_{}.csv".format(chrom, self.sample_id) for chrom in self.chromosomes], 
                    dtype=dtype, usecols=usecols), ignore_index=True)
        col_names = self.df.columns.values.tolist()
        new_col_names = [''.join(char for char in name if char.isalnum()) for name in col_names]
        self.df.rename(columns=dict(zip(col_names, new_col_names)))

In [21]:
%%add_to Mech
def category_filter(self, **kwargs):
    for key, value in kwargs.items():
        cats = self.df[key].cat.categories
        self.df.remove_categories([x for x in cats if not x in value])
    self.df.dropna(inplace=True)

In [22]:
%%add_to Mech
def num_filter(self, **kwargs):
    pd.eval("DP = self.df.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=self.df, inplace=True)
    pd.eval("AF = self.df.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=self.df, inplace=True)
    for key, value in kwargs.items():
        self.df.loc[:,key].where(self.df.loc[:,key] > value, inplace=True)
    self.df.dropna(inplace=True)

In [24]:
def find_mid(indices):
    copy = indices[:-1].copy()
    np.insert(copy, 0, indices[0] - 1)
    return copy

In [25]:
def inv_arr(max_val, idx_arr):
    arr = set(range(0,max_val))
    diff = arr.difference(idx_arr)
    return diff

In [ ]:
class Uqid():
    def __init__(self, series, len1, len2):
        self.series = series
        self.len1 = len1
        self.len2 = len2

In [29]:
%%add_to Uqid
    def create_idx_col(self):
        self.idxs = np.append(np.arange(0, self.len1), np.arange(0, self.len2))
        self.df = pd.DataFrame({'idx':idx}, index=self.series)
        self.df.reset_index(inplace=True)

In [ ]:
%%add_to Uqid
    def filter_uqs(self):
        self.df = self.df.sort_index()
        self.df = self.df.groupby('index')
        self.df = self.df.filter(lambda x: len(x) == 1)

In [ ]:
def combine_uqids(mech1_,mech2_):
    uqid1 = mech1_.df[mech1_.df.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()
    uqid2 = mech2_.df[mech2_.df.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()    
    combined_uqids = pd.concat([uqid1, uqid2], ignore_index=True)
    combine = Uqid(combined_uqids, len(uqid1.index), len(uqid2.index))
    return combine

In [30]:
def subtract(mech1, mech2):
    combine_mechs = combine_uqids(mech1_,mech2_)
    combine_mechs.create_idx_col()
    combine_mechs.filter_uqs()
    idx = combine_mechs.df['idx'].to_numpy()
    mid_val = find_mid(idx)
    drop_vals = inv_arr(combine_mechs.len1, idx[:mid_val])
    uq_df = mech1.df.drop(mech1.df.index[drop_vals])
    return uq_df

In [31]:
def filter(**kwargs):
    return None

In [34]:
def main():
    
    # CHANGE THESE VALUES BELOW
    ids = {'22241':['22334','22243']}
    chromosomes = ['1','2']
    min_params = dict(DP=15, AF=.15, Otherinfo10=['PASS'])
    sub_params = dict()
    # CHANGE THESE VALUES ABOVE
    
    for key, value in ids.items():
        min_parent = Mech(chromosomes, key).filter(**min_params)
        sub_parent = Mech(chromosomes, key).filter(**sub_params)
        for val in value:
            min_variant = Mech(chromosomes, val).filter(**min_params)
            sub_variant = Mech(chromosomes, val).filter(**sub_params)
            
            subtract(min_parent, sub_variant).to_csv("{}-{}.csv".format(key, val))
            subtract(min_variant, sub_parent).to_csv("{}-{}.csv".format(val, key))

In [ ]:
main()